In [1]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [2]:
img1 = cv.imread('./input/0/611269920_0.jpg',cv.IMREAD_GRAYSCALE)
img2 = cv.imread('./input/1/611269920_1.jpg',cv.IMREAD_GRAYSCALE)
# Initiate SIFT detector
sift = cv.SIFT_create(nfeatures=5000, contrastThreshold=0.00001, edgeThreshold=10)
# find the keypoints and descriptors with SIFT
kp1, des1 = sift.detectAndCompute(img1,None)
kp2, des2 = sift.detectAndCompute(img2,None)
# BFMatcher with default params
bf = cv.BFMatcher()
matches = bf.knnMatch(des1,des2,k=2)
# Apply ratio test
good = []
for m,n in matches:
    if m.distance < 0.75*n.distance:
        good.append([m])
# cv.drawMatchesKnn expects list of lists as matches.
img3 = cv.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)
cv.imwrite('./test.jpg', img3)

True

In [3]:
sift_detector = cv.SIFT_create(nfeatures=5000, nOctaveLayers=10, edgeThreshold=100, enable_precise_upscale=True)  # SIFT with increased number of features
orb_detector = cv.ORB_create(nfeatures=5000)    # ORB with increased number of features
akaze_detector = cv.AKAZE_create()              # AKAZE with default settings

# Function to detect and draw keypoints using different detectors
def detect_features(image, detector):
    keypoints, des = detector.detectAndCompute(image, None)
    keypoint_image = cv.drawKeypoints(image, keypoints, None, flags=cv.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
    return keypoint_image, keypoints, des

# Detect features using SIFT
sift_keypoint_image1, sift_keypoints1, sift_des1 = detect_features(img1, sift_detector)
sift_keypoint_image2, sift_keypoints2, sift_des2 = detect_features(img2, sift_detector)

# Detect features using ORB
orb_keypoint_image1, orb_keypoints1, orb_des1 = detect_features(img1, orb_detector)
orb_keypoint_image2, orb_keypoints2, orb_des2 = detect_features(img2, orb_detector)

# Detect features using AKAZE
akaze_keypoint_image1, akaze_keypoints1, _ = detect_features(img1, akaze_detector)
akaze_keypoint_image2, akaze_keypoints2, _ = detect_features(img2, akaze_detector)

# Save the images with keypoints detected by different detectors
sift_output_path1 = './sift_keypoints1.jpg'
cv.imwrite(sift_output_path1, sift_keypoint_image1)
sift_output_path2 = './sift_keypoints2.jpg'
cv.imwrite(sift_output_path2, sift_keypoint_image2)

orb_output_path1 = './orb_keypoints1.jpg'
cv.imwrite(orb_output_path1, orb_keypoint_image1)
orb_output_path2 = './orb_keypoints2.jpg'
cv.imwrite(orb_output_path2, orb_keypoint_image2)

akaze_output_path1 = './akaze_keypoints1.jpg'
cv.imwrite(akaze_output_path1, akaze_keypoint_image1)
akaze_output_path2 = './akaze_keypoints2.jpg'
cv.imwrite(akaze_output_path2, akaze_keypoint_image2)

True

In [4]:
bf = cv.BFMatcher()

matches = bf.knnMatch(sift_des1, sift_des2, k=2)
# Sort them in the order of their distance.

good = []
for m,n in matches:
    if m.distance < 0.8*n.distance:
        good.append([m])
# cv.drawMatchesKnn expects list of lists as matches.
img3 = cv.drawMatchesKnn(img1,sift_keypoints1,img2,sift_keypoints2,good,None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

In [5]:
cv.imwrite("./test.jpg", img3)

True

In [6]:
bf = cv.BFMatcher(cv.NORM_HAMMING, crossCheck=True)
# Match descriptors.
matches = bf.match(orb_des1,orb_des2)
# Sort them in the order of their distance.
matches = sorted(matches, key = lambda x:x.distance)
# Draw first 10 matches.
img4 = cv.drawMatches(img1,orb_keypoints1,img2,orb_keypoints2,matches[:10],None,flags=cv.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS)

cv.imwrite("./test2.jpg",img4)

True

In [7]:
# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary
flann = cv.FlannBasedMatcher(index_params,search_params)
matches = flann.knnMatch(sift_des1,sift_des2,k=2)
# Need to draw only good matches, so create a mask
matchesMask = [[0,0] for i in range(len(matches))]
# ratio test as per Lowe's paper
for i,(m,n) in enumerate(matches):
    if m.distance < 0.8*n.distance:
        matchesMask[i]=[1,0]
draw_params = dict(matchColor = (0,255,0),
                   singlePointColor = (255,0,0),
                   matchesMask = matchesMask,
                   flags = cv.DrawMatchesFlags_DEFAULT)
img5 = cv.drawMatchesKnn(img1,sift_keypoints1,img2,sift_keypoints2,matches,None,**draw_params)

cv.imwrite("./test3.jpg", img5)

True